## 1. Importación de librerías requeridas

Importaremos las librerías `pandas` y `scikit-learn`. En particular, usaremos las siguientes clases y funciones:

* `GridSearchCV`: clase para entrenar múltiples modelos variando sus parámetros. Se utiliza para hacer una búsqueda exhaustiva de los mejores valores para el entrenamiento de un modelo.
* `KFold`: clase para definir múltiples conjuntos de entrenamiento y validación sobre nuestro conjunto de datos.
* `MinMaxScaler`, `RobustScaler` y `StandardScaler`: clases para escalar los valores de nuestro conjunto de datos.
* `PolynomialFeatures`: genera un nuevo conjunto de variables que representa todas las combinaciones polinomiales de grado menor o igual a un grado especificado como parámetro.
* `make_pipeline`: función para definir una secuencia de pasos para transformar datos y entrenar modelos.

In [34]:

import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures,MinMaxScaler, StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline

from importlib_metadata import version

print(f"Version de Pandas: {version('pandas')}")
print(f"Version de Scikit-learn:  {version("scikit-learn")}")

Version de Pandas: 2.3.1
Version de Scikit-learn:  1.7.1


## 2. Carga de datos

Realizaremos la carga de datos usando la función de Pandas `read_csv()` y especificando el separador del archivo:

In [35]:
data_raw = pd.read_csv("./kc_house_data.csv")

In [36]:
data_raw.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## 3. Preparación de los datos

Utilizaremos la variable `data` para almacenar un conjunto de datos modificado. En ese sentido, podremos tener una copia de los datos originales almacenada en la variable `data_raw`, por si en algún momento es necesario recuperar información que haya sido modificada:

In [41]:
data = data_raw.copy()

### Eliminación de variables poco relevantes

Eliminaremos tres variables con poca relevancia para el precio de las viviendas: `id`, `date` y `zipcode`, haciendo uso de la función `drop()` sobre nuestro DataFrame:

In [42]:
data = data.drop(['id','date','zipcode'], axis="columns")

In [43]:
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503


### División de datos

Ahora dividiremos el conjunto de datos resultante en un conjunto de entrenamiento y uno de pruebas mediante la función `train_test_split()`. Usaremos el 80% de los datos para el entrenamiento y el 20% restante para las pruebas:

In [44]:
train , test = train_test_split(data,test_size=0.2,random_state=9)
train.head()
 

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
17690,1364000.0,4,2.50,3560,8960,2.0,0,0,3,10,3560,0,2001,0,47.6903,-122.213,1660,7680
7711,815000.0,3,2.50,2415,2186,2.0,0,1,3,9,2415,0,1981,0,47.6506,-122.202,2660,2165
7878,205000.0,4,1.00,1030,6621,1.0,0,0,4,6,1030,0,1955,0,47.4857,-122.221,1420,6631
5409,439950.0,4,2.25,2780,15075,2.0,0,0,3,7,2780,0,1985,0,47.4770,-122.116,1650,25542
10047,362362.0,2,1.00,710,4000,1.0,0,0,3,6,710,0,1909,0,47.5535,-122.269,960,4000


Como los algoritmos supervisados implementados en `scikit-learn` necesitan que las variables de entrada estén separadas de la variable objetivo, usaremos la función `drop` y definiremos `x_train` y `y_train`, que representan los valores de las variables independientes y los valores de la variable objetivo, respectivamente:

In [45]:
x_train = train.drop(['price'], axis="columns")
y_train = train["price"]

In [14]:

# # Verificación
# print('Tamaños: ')
# print('\tDataset original: ', x.shape, y.shape)
# print('\tEntrenamiento: ', x_train.shape, y_train.shape)
# print('\tPrueba: ', x_test.shape, y_test.shape)

# print('Proporciones categorías (0s/1s): ')
# print(f'\tDataset original: {np.sum(y==0)/len(y)}/{np.sum(y==1)/len(y)}')
# print(f'\tEntrenamiento: {np.sum(y_train==0)/len(y_train)}/{np.sum(y_train==1)/len(y_train)}')
# print(f'\tPrueba: {np.sum(y_test==0)/len(y_test)}/{np.sum(y_test==1)/len(y_test)}')

### Estandarización

Para el caso de regresión regularizada, la escala en la que se encuentran las variables se vuelve relevante, a diferencia de la regresión lineal simple. Específicamente, dado que los métodos de regularización actúan sobre la magnitud de los coeficientes del modelo, todos deben estar en la misma escala. Es por eso que utilizaremos un objeto de la clase `MinMaxScaler()`.

En la variable `scaler` asignamos el objeto que corresponde al escalador para las variables numéricas, en Scikit-learn hay varios y los más comunes son los siguientes:

* **StandardScaler**: Escala las variables para que la distribución de probabilidad de cada variable se comporte como una distribución normal (es decir, media 0 y varianza 1)
* **MinMaxScaler**: Escala las variables en un rango especificado (que es [0,1] por defecto y puede definirse manualmente).
* **RobustScaler**: Remueve la media y escala las variables usando el rango interquartil. Este es útil cuando las variables tienen valores atípicos (_outliers_)

#### Experimento

Intenta cambiar el escalador de la celda siguiente usando `StandardScaler()` o `RobustScaler()` y revisa qué efectos tiene cada uno en los coeficientes resultantes en la regresión y en el rendimiento del modelo. Diferentes escaladores tendrán mejores o peores resultados dependiendo de los datos.

Si deseas ir más allá, puedes agregar el escalado de variables a la fase de búsqueda de hiperparámetros.

In [46]:
scaler = MinMaxScaler()

Este objeto tiene el método `fit_transform()` para realizar la estandarización, pero retorna un arreglo en vez de un DataFrame. Por lo tanto, primero almacenaremos la información de las columnas en la variable `columns` y, finalmente, reconstruiremos el DataFrame:

In [47]:
columns = x_train.columns
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train, columns=columns)
x_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,0.121212,0.31250,0.242217,0.005113,0.4,0.0,0.00,0.50,0.7,0.352876,0.000000,0.878261,0.000000,0.859579,0.254153,0.220802,0.008074
1,0.090909,0.31250,0.155277,0.001009,0.4,0.0,0.25,0.50,0.6,0.226217,0.000000,0.704348,0.000000,0.795721,0.263289,0.395903,0.001739
2,0.121212,0.12500,0.050114,0.003696,0.0,0.0,0.00,0.75,0.3,0.073009,0.000000,0.478261,0.000000,0.530481,0.247508,0.178778,0.006869
3,0.121212,0.28125,0.182992,0.008817,0.4,0.0,0.00,0.50,0.4,0.266593,0.000000,0.739130,0.000000,0.516487,0.334718,0.219051,0.028592
4,0.060606,0.12500,0.025816,0.002108,0.0,0.0,0.00,0.50,0.3,0.037611,0.000000,0.078261,0.000000,0.639537,0.207641,0.098231,0.003847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,0.121212,0.40625,0.233865,0.006494,0.4,0.0,0.75,0.75,0.7,0.227876,0.246973,0.521739,0.993052,0.482065,0.122093,0.352478,0.021651
17286,0.090909,0.25000,0.079727,0.002156,0.2,0.0,0.00,1.00,0.5,0.116150,0.000000,0.034783,0.000000,0.843172,0.142027,0.143758,0.002767
17287,0.090909,0.31250,0.156416,0.003147,0.4,0.0,0.00,0.50,0.4,0.227876,0.000000,0.860870,0.000000,0.347274,0.291528,0.462441,0.005802
17288,0.060606,0.12500,0.085042,0.002002,0.0,0.0,0.00,0.50,0.4,0.054204,0.152542,0.252174,0.000000,0.842850,0.114618,0.185782,0.003646


Visualizaremos los nuevos valores del conjunto de entrenamiento usando head():

In [48]:
x_train.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,0.121212,0.31250,0.242217,0.005113,0.4,0.0,0.00,0.50,0.7,0.352876,0.0,0.878261,0.0,0.859579,0.254153,0.220802,0.008074
1,0.090909,0.31250,0.155277,0.001009,0.4,0.0,0.25,0.50,0.6,0.226217,0.0,0.704348,0.0,0.795721,0.263289,0.395903,0.001739
2,0.121212,0.12500,0.050114,0.003696,0.0,0.0,0.00,0.75,0.3,0.073009,0.0,0.478261,0.0,0.530481,0.247508,0.178778,0.006869
3,0.121212,0.28125,0.182992,0.008817,0.4,0.0,0.00,0.50,0.4,0.266593,0.0,0.739130,0.0,0.516487,0.334718,0.219051,0.028592
4,0.060606,0.12500,0.025816,0.002108,0.0,0.0,0.00,0.50,0.3,0.037611,0.0,0.078261,0.0,0.639537,0.207641,0.098231,0.003847


## 4. Entrenamiento de un modelo de referencia

Con el conjunto de datos modificado, empezaremos con el entrenamiento de un modelo de referencia, que nos permitirá ver cómo es el desempeño de un modelo de regresión lineal simple sobre este conjunto de datos. Definiremos un objeto de tipo `LinearRegression()` y lo entrenaremos con la función `fit()`, utilizando el conjunto de entrenamiento separado en las variables independientes `x_train` y la variable objetivo `y_train`:

In [49]:
reg_lineal = LinearRegression().fit(x_train, y_train)

Ahora veremos los coeficientes y el intercepto resultantes:

In [31]:
print ('Coeficientes: ', reg_lineal.coef_)
print ('Intercepto: ', reg_lineal.intercept_)

Coeficientes:  [-1045667.82788789   349811.79988406   800497.55201274   242049.55919058
     4639.74845105   584669.85722034   201276.17419331   127355.53285441
   961197.84442804  1020275.34410273   319434.29765598  -289152.79106719
    46639.14802456   348546.72281543  -144037.33147272   184630.22563774
  -362614.18447602]
Intercepto:  -335986.24078267254


Podemos ver qué coeficiente le corresponde a cada variable con el método zip(), usando los nombres de las variables almacenados en x_train.columns:

In [50]:
pd.DataFrame(zip(x_train.columns, reg_lineal.coef_),columns=["Variable","Coeficiente"])

,Variable,Coeficiente
0,bedrooms,-1.045668e+06
1,bathrooms,3.498118e+05
2,sqft_living,8.004976e+05
3,sqft_lot,2.420496e+05
4,floors,4.639748e+03
5,waterfront,5.846699e+05
6,view,2.012762e+05
7,condition,1.273555e+05
8,grade,9.611978e+05
9,sqft_above,1.020275e+06


### Evaluación del modelo

Ahora utilizaremos el conjunto de pruebas para evaluar el desempeño del modelo. Separaremos las variables independientes y la variable objetivo, de la misma forma que para el conjunto de entrenamiento:

In [51]:
x_test = test.drop(['price'],axis="columns")
y_test = test['price']

También usaremos la variable `scaler` para escalar las variables independientes del conjunto de pruebas. Ten en cuenta que solo utilizaremos la información de las variables disponible en el conjunto de entrenamiento, por lo que haremos uso del método `transform()`:

In [52]:
x_test = scaler.transform(x_test)
x_test = pd.DataFrame(x_test, columns=columns)

Finalmente, realizaremos las predicciones. Utilizaremos tres métricas para evaluar el desempeño del modelo de referencia: la raíz del error cuadrático medio, el error absoluto medio y el coeficiente de determinación: 

In [53]:
y_pred = reg_lineal.predict(x_test)

print(f'------ Modelo de regresión lineal simple----')
print(f"RMSE: {root_mean_squared_error(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f'R²: {r2_score(y_test, y_pred):.2f}')

------ Modelo de regresión lineal simple----
RMSE: 200731.58
MAE: 126120.34
R²: 0.71


Como puedes observar, el modelo de regresión simple se ajusta un 71% a los datos, aunque se tienen errores medios en el orden de los cientos de miles de dólares.

## 5. Búsqueda de hiperparámetros

Cambiar el valor de los hiperparámetros tiene un impacto directo sobre el desempeño del modelo resultante, por lo que queremos encontrar un valor que resulte en el mejor desempeño posible. A continuación, ejecutaremos un procedimiento de búsqueda exhaustiva, es decir, vamos a entrenar modelos con varios valores para `alpha` y, al finalizar, nos quedaremos con el valor del hiperparámetro que resulte en el modelo con el mejor desempeño.

Lo primero que haremos es definir un objeto de tipo `Lasso()`, que puede recibir un valor del hiperparámetro `alpha`. En este caso, al cambiar su valor durante la búsqueda, no es necesario especificarlo. Sin embargo, vamos a definir el parámetro `max_iter` para aumentar el número de iteraciones y, de esa forma, ayudar a la convergencia del modelo:

In [56]:
from sklearn.linear_model import Lasso
lasso = Lasso(max_iter=500)



Ahora utilizaremos un diccionario para definir nuestro espacio de búsqueda de hiperparámetros, es decir, los valores que vamos a probar y sobre los que decidiremos cuál escoger. Almacenaremos estos valores en la variable `param_grid`:

In [59]:
param_grid = {'alpha': [1, 2, 5, 10, 15, 20]}
param_grid

{'alpha': [1, 2, 5, 10, 15, 20]}

Además vamos a definir un objeto de la clase `KFold()`, que nos será útil para obtener una estimación del desempeño del modelo más realista que simplemente utilizando el conjunto de pruebas. La validación cruzada k-fold es un método que toma el conjunto de entrenamiento original y lo separa en k grupos, usando uno como validación y el resto (k-1) como entrenamiento. Después de definir los grupos, entrena el modelo con el conjunto de entrenamiento y lo evalúa con el conjunto de validación, repitiendo este proceso para cada uno de los k grupos. En este caso, definiremos `k=5` y usaremos el parámetro `shuffle=True` para indicar que se cambie el orden de los datos antes de separarlos en los grupos:

In [60]:
kfold = KFold(n_splits=5, shuffle=True, random_state = 0)

A continuación, vamos a utilizar `GridSearchCV` para realizar la búsqueda exhaustiva del mejor hiperparámetro. Definiremos el algoritmo `lasso`, los valores del hiperparámetro `param_grid` y la estrategia de validación cruzada `kfold`.

* La complejidad del modelo afecta el tiempo que tarda en entrenar, por lo que esta parte puede tardar más tiempo en completarse, puedes intentar agregar el parámetro `n_jobs=-1` para mejorar el rendimiento o usar `n_jobs=1` para limitarlo en caso de que el entorno falle

In [61]:
modelos_grid = GridSearchCV(lasso, param_grid, cv=kfold, n_jobs=-1,verbose=2)

Finalmente, entrenaremos los modelos con los conjuntos definidos previamente (**Nota:** el entrenamiento puede tardar algunos minutos):

* la línea `%%time` permite ver cuánto tiempo tardó en ejecutarse

In [62]:
%%time
modelos_grid.fit(x_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/Users/johanypenaflorez/miniforge3/envs/ML_ENV/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.590e+13, tolerance: 1.857e+11
  model = cd_fast.enet_coordinate_descent(
/Users/johanypenaflorez/miniforge3/envs/ML_ENV/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.280e+13, tolerance: 1.845e+11
  model = cd_fast.enet_coordinate_descent(
/Users/johanypenaflorez/miniforge3/envs/ML_ENV/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, chec

[CV] END ............................................alpha=1; total time=   0.1s
[CV] END ............................................alpha=1; total time=   0.1s
[CV] END ............................................alpha=1; total time=   0.1s
[CV] END ............................................alpha=2; total time=   0.1s
[CV] END ............................................alpha=1; total time=   0.1s
[CV] END ............................................alpha=2; total time=   0.1s
[CV] END ............................................alpha=2; total time=   0.1s
[CV] END ............................................alpha=1; total time=   0.1s
[CV] END ............................................alpha=2; total time=   0.1s
[CV] END ............................................alpha=2; total time=   0.1s
[CV] END ............................................alpha=5; total time=   0.1s
[CV] END ...........................................alpha=10; total time=   0.1s
[CV] END ...................

/Users/johanypenaflorez/miniforge3/envs/ML_ENV/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.224e+13, tolerance: 1.828e+11
  model = cd_fast.enet_coordinate_descent(
/Users/johanypenaflorez/miniforge3/envs/ML_ENV/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.375e+13, tolerance: 1.849e+11
  model = cd_fast.enet_coordinate_descent(
/Users/johanypenaflorez/miniforge3/envs/ML_ENV/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, chec

[CV] END ...........................................alpha=15; total time=   0.1s
[CV] END ...........................................alpha=20; total time=   0.1s
[CV] END ...........................................alpha=15; total time=   0.1s
[CV] END ...........................................alpha=15; total time=   0.1s
[CV] END ...........................................alpha=15; total time=   0.1s
[CV] END ...........................................alpha=20; total time=   0.1s
[CV] END ...........................................alpha=20; total time=   0.1s
[CV] END ...........................................alpha=20; total time=   0.1s
[CV] END ...........................................alpha=20; total time=   0.2s
CPU times: user 317 ms, sys: 797 ms, total: 1.11 s
Wall time: 3.4 s


/Users/johanypenaflorez/miniforge3/envs/ML_ENV/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.203e+13, tolerance: 2.308e+11
  model = cd_fast.enet_coordinate_descent(


,estimator,Lasso(max_iter=500)
,param_grid,"{'alpha': [1, 2, ...]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,alpha,5


Obtendremos el mejor parámetro usando el atributo best_params_:

In [63]:
print("Mejor parámetro: ", modelos_grid.best_params_)

Mejor parámetro:  {'alpha': 5}


Como puedes ver, el mejor valor del hiperparámetro `alpha` es 5. Podemos obtener el mejor modelo haciendo uso del atributo `best_estimator_`, y obtendremos sus coeficientes usando el atributo `coef_`:

In [64]:
mejor_modelo = modelos_grid.best_estimator_
pd.DataFrame(zip(x_train.columns, mejor_modelo.coef_),columns=["Variable","Coeficiente"])

,Variable,Coeficiente
0,bedrooms,-1.036039e+06
1,bathrooms,3.456999e+05
2,sqft_living,2.612576e+06
3,sqft_lot,2.192080e+05
4,floors,5.003721e+03
5,waterfront,5.838947e+05
6,view,2.016535e+05
7,condition,1.269526e+05
8,grade,9.633326e+05
9,sqft_above,-2.216832e+05


### Evaluación del mejor modelo

A continuación realizaremos predicciones sobre el conjunto de pruebas para comparar con los valores de `y_test`. Utilizaremos la función `predict()` sobre el mejor modelo, y obtendremos tres métricas de desempeño:

In [65]:
y_pred = mejor_modelo.predict(x_test)

print(f'------ Modelo de regresión Lasso ----')
print(f"RMSE: {root_mean_squared_error(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f'R²: {r2_score(y_test, y_pred):.2f}')

------ Modelo de regresión Lasso ----
RMSE: 200717.60
MAE: 126119.77
R²: 0.71


Con respecto al modelo de regresión lineal simple, el valor de R2, así como el valor de la raíz del error cuadrático medio y del error absoluto medio se mantienen similares. Es decir, para este conjunto de datos, el rendimiento de generalización no presenta mejorías con regularización L1 y un valor de alpha de 5.

# 6. Entrenamiento de un modelo de selección de variables

Si se entrena un modelo de regresión Lasso con un valor alto de `alpha`, la regularización puede llegar a anular los coeficientes de algunas variables. Este comportamiento permite realizar una selección de las variables menos importantes, anulando su influencia en las predicciones de la variable objetivo. Usando nuestro conjunto de datos, vamos a definir un objeto de tipo `Lasso()`, pero esta vez utilizando un valor de `alpha` de 100:

In [66]:
lasso_sv = Lasso(alpha=100)

Entrenaremos el modelo utilizando el conjunto de entrenamiento:

In [67]:
%%time
lasso_sv.fit(x_train, y_train)

CPU times: user 48.4 ms, sys: 124 ms, total: 172 ms
Wall time: 87.1 ms


,alpha,100
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


Y obtendremos los coeficientes resultantes:

In [69]:
pd.DataFrame(zip(x_train.columns, lasso_sv.coef_),columns=["Variable","Coeficiente"])

,Variable,Coeficiente
0,bedrooms,-7.825018e+05
1,bathrooms,3.291499e+05
2,sqft_living,1.727476e+06
3,sqft_lot,0.000000e+00
4,floors,5.366111e+03
5,waterfront,5.722813e+05
6,view,2.073721e+05
7,condition,1.214180e+05
8,grade,9.798325e+05
9,sqft_above,3.183705e+05


Como puedes observar, el coeficiente de algunas variables es 0, lo que implica que este modelo no tiene en cuenta estas variables para realizar predicciones. Una de las particularidades de la regresión Lasso es que, gracias al uso del término de penalización, es capaz de determinar variables que no son relevantes para la estimación de la variable objetivo. Usualmente esta selección de variables se produce con valores grandes del hiperparámetro alpha, lo que implica que la regularización también es mucho mayor. Este resultado puede ser utilizado para, por ejemplo, establecer las variables más relevantes y entrenar un modelo diferente sobre el subconjunto de variables obtenido.

## Cierre

En este tutorial hemos utilizado nuevas clases de scikit-learn para entrenar modelos de regresión Lasso. Adicionalmente, observamos cómo estandarizar datos, definimos un objeto para realizar validación cruzada con k-Folds y realizamos una búsqueda exhaustiva de hiperparámetros. Finalmente, observamos cómo se puede utilizar la regresión Lasso para realizar una selección de variables del conjunto de datos.

---
Si quieres más información sobre regresión Lasso en `scikit_learn` puedes consultar el [sitio web oficial](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

Para la estandarización de datos con la clase `MinMaxScaler()` puedes consultar [este enlace](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

En particular, puedes ver el efecto que tiene cada escalador de Scikit-learn en [este ejemplo de scikit-learn](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#)

Para la búsqueda de hiperparámetros con la clase `GridSearchCV()` puedes consultar [este enlace](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

Finalmente, para obtener más información sobre la clase `KFold()` puedes ir [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

---
*Creado por: Nicolás Díaz*

*Última edición: Camilo Rozo*

*Revisado por: Haydemar Nuñez*

*Versión: Enero 2025*  

*Universidad de los Andes*  